In [1]:
import pandas as pd
import numpy as np
import os
import ast

In [2]:
# Replace 'your_directory_path' with the path to your folder
folder_path = 'atbat'

# List all CSV files in the folder
csv_files = [file for file in os.listdir(folder_path) if file.endswith('.csv')]


In [3]:
csv_files

['2020.csv',
 '2021.csv',
 '2023.csv',
 '2022.csv',
 '2019.csv',
 '2018.csv',
 '2024.csv',
 '2015.csv',
 '2014.csv',
 '2016.csv',
 '2017.csv']

In [4]:
final = pd.DataFrame()

for file in csv_files:
    
    data = pd.read_csv(f'atbat/{file}')
    
    final = pd.concat([data, final])

In [5]:
final = final.sort_values(by='Date')
final.to_csv('atbats.csv')

In [6]:
final.drop(columns=['Unnamed: 0'], inplace=True)

In [7]:
final.head()

,Outcome,Rbi,Bases,Batter_id,Batter_name,Pitcher,Isout,Outs,homeScore,awayScore,inning,isTopInning,runner_pos,dict_players,Date
644,strikeout,0,Empty,449107,Mike Aviles,vs_LHP,True,2,4,3,8,True,[0],{'init': 0},2014-04-01
435,home_run,3,RISP,474832,Brandon Belt,vs_LHP,False,2,0,4,1,True,"[0, 2, 1]","{'init': 0, 467055: 2, 457763: 1}",2014-04-01
436,field_out,0,Empty,435078,Joaquin Arias,vs_LHP,True,3,0,4,1,True,[0],{'init': 0},2014-04-01
437,single,0,Empty,467827,Gerardo Parra,vs_RHP,False,0,0,4,1,False,[0],{'init': 0},2014-04-01
438,field_out,0,Men_On,431094,Aaron Hill,vs_RHP,True,1,0,4,1,False,"[0, 1]","{'init': 0, 467827: 1}",2014-04-01


In [8]:
final['Double_Play'] = final['Outcome'].apply(lambda x: 1 if 'double_play' in x else 0)

In [10]:
final['Current_outs'] = final.apply(lambda x: x['Outs'] - 2 if (x['Isout'] == True) & (x['Double_Play'] == 1) else (1 if (x['Isout'] == True) else x['Outs']), axis=1)
final['runner_pos'] = final['runner_pos'].apply(lambda x: ast.literal_eval(x))

final['Solo_runner'] = final['runner_pos'].apply(lambda x: 1 if len(x) == 2 else 0)
final['Highest_Base'] = final['runner_pos'].apply(lambda x: max(x))
final['Scoring_pos'] = final['Bases'].apply(lambda x: 1 if x == 'RISP' else 0)

In [15]:
second_rbi = ['double', 'single', 'triple', 'home_run']

third_rbi = ['double', 'single', 'triple', 'home_run', 'sac_fly', 'sac_bunt', 'sac_fly_double_play', 'sac_bunt_double_play']

first_rbi = ['triple', 'home_run']

final['Rbi_outcome_second'] = final['Outcome'].apply(lambda x: 1 if x in second_rbi else 0)
final['Rbi_outcome_third'] = final['Outcome'].apply(lambda x: 1 if x in third_rbi else 0)
final['Rbi_outcome_first'] = final['Outcome'].apply(lambda x: 1 if x in first_rbi else 0)

In [16]:
filter_df_third = final.loc[(final['Solo_runner'] == 1) & (final['Highest_Base'] == 3)]

filter_df_second = final.loc[(final['Solo_runner'] == 1) & (final['Highest_Base'] == 2)]

filter_df_first = final.loc[(final['Solo_runner'] == 1) & (final['Highest_Base'] == 1)]

In [17]:
men_on = final.loc[(final['Highest_Base'] == 2)]

In [18]:
occurences = []

for row, i in men_on.iterrows():
    
    date = i['Date']
    
    batter = i['Batter_name']
    
    past_df = filter_df_third.loc[(filter_df_third['Date'] < date) & (filter_df_third['Batter_name'] == batter)].sort_values(by='Date').tail(20)
    
    if len(past_df) < 20:
        continue

    rbis = (past_df['Rbi'] != 0).sum()

    successful = rbis / len(past_df)
    
    past_df = filter_df_second.loc[(filter_df_second['Date'] < date) & (filter_df_second['Batter_name'] == batter)].sort_values(by='Date').tail(20)
    
    if len(past_df) < 20:
        continue

    rbis = (past_df['Rbi'] != 0).sum()

    successful_stay = rbis / len(past_df)
    
    
    occurences.append({
        'Batter_name': batter,
        'Rbi_outcome': i['Rbi_outcome_third'],
        'Rbi_stay': i['Rbi_outcome_second'],
        'Success_steal': successful,
        'Success_stay': successful_stay,
        'Outs': i['Outs'],
        'homeScore': i['homeScore'],
        'awayScore': i['awayScore'],
        'inning': i['inning'],
        'isTopInning': i['isTopInning'],
        'Isout': i['Isout'],
        'Current_outs': i['Current_outs'],
        'dict_players':i['dict_players'],
        'Date': i['Date']
    })

In [19]:
occurences_df = pd.DataFrame(occurences)
occurences_df.to_csv('atbats_second_multi.csv')

In [ ]:
men_on = final.loc[(final['Solo_runner'] == 1) & (final['Highest_Base'] == 1)]

In [ ]:
occurences = []

for row, i in men_on.iterrows():
    
    date = i['Date']
    
    batter = i['Batter_name']
    
    past_df = filter_df_second.loc[(filter_df_second['Date'] < date) & (filter_df_second['Batter_name'] == batter)].sort_values(by='Date').tail(20)
    
    if len(past_df) < 20:
        continue

    rbis = (past_df['Rbi'] != 0).sum()

    success = rbis / len(past_df)
    
    past_df = filter_df_first.loc[(filter_df_first['Date'] < date) & (filter_df_first['Batter_name'] == batter)].sort_values(by='Date').tail(20)
    
    if len(past_df) < 20:
        continue

    rbis = (past_df['Rbi'] != 0).sum()

    success_stay = rbis / len(past_df)
    
    
    occurences.append({
        'Batter_name': batter,
        'Rbi_outcome': i['Rbi_outcome_second'],
        'Rbi_stay': i['Rbi_outcome_first'],
        'Success_steal': successful,
        'Success_stay': success_stay,
        'Outs': i['Outs'],
        'homeScore': i['homeScore'],
        'awayScore': i['awayScore'],
        'inning': i['inning'],
        'isTopInning': i['isTopInning'],
        'Isout': i['Isout'],
        'Current_outs': i['Current_outs'],
        'dict_players':i['dict_players'],
        'Date': i['Date']
    })

In [ ]:
occurences_df = pd.DataFrame(occurences)
occurences_df.to_csv('atbats_first_solo.csv')

In [65]:
final['Highest_Base'] = final['runner_pos'].apply(lambda x: max(ast.literal_eval(x)))

In [69]:
sec_df = final.loc[(final['Highest_Base'] == 2) & (final['Current_outs'] <= 1)]

In [82]:
filter_df = final.loc[(final['Highest_Base'] ==3) & (final['Current_outs'] <=1)]

In [ ]:
occurences = []

for row, i in sec_df.iterrows():
    
    date = i['Date']
    
    batter = i['Batter_name']
    
    past_df = filter_df.loc[(filter_df['Date'] < date) & (filter_df['Batter_name'] == batter)].sort_values(by='Date').tail(10)
    
    if len(past_df) < 10:
        continue

    rbis = (past_df['Rbi'] != 0).sum()

    successful = rbis / len(past_df)
    
    
    occurences.append({
        'Batter_name': batter,
        'Rbi_outcome': i['Rbi'],
        'Success_past': successful,
        'Outs': i['Outs'],
        'homeScore': i['homeScore'],
        'awayScore': i['awayScore'],
        'inning': i['inning'],
        'isTopInning': i['isTopInning'],
        'Isout': i['Isout'],
        'Current_outs': i['Current_outs'],
    })

In [42]:
test = final.loc[final['Batter_name'] == 'Buster Posey']

In [43]:
test

,Outcome,Rbi,Bases,Batter_id,Batter_name,Pitcher,Isout,Outs,homeScore,awayScore,inning,isTopInning,runner_pos,Date,Current_outs,Scoring_pos
433,single,1,RISP,457763,Buster Posey,vs_LHP,False,1,0,1,1,True,"[0, 2, 1]",2014-04-01,1,1
474,walk,0,Empty,457763,Buster Posey,vs_LHP,False,2,3,4,6,True,[0],2014-04-01,2,0
453,field_out,0,Empty,457763,Buster Posey,vs_LHP,True,3,2,4,3,True,[0],2014-04-01,2,0
493,strikeout,0,Men_On,457763,Buster Posey,vs_RHP,True,2,5,4,8,True,"[0, 1]",2014-04-01,1,0
1744,single,0,Empty,457763,Buster Posey,vs_RHP,False,1,0,1,6,True,[0],2014-04-02,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183004,single,0,Empty,457763,Buster Posey,vs_RHP,False,1,0,0,5,False,[0],2021-09-29,1,0
183983,field_out,0,Empty,457763,Buster Posey,vs_RHP,True,1,4,4,8,False,[0],2021-09-30,0,0
183945,sac_fly,1,RISP,457763,Buster Posey,vs_LHP,True,1,2,3,3,False,"[0, 3, 1]",2021-09-30,0,1
183962,double,1,Men_On,457763,Buster Posey,vs_LHP,False,1,4,4,5,False,"[0, 1]",2021-09-30,1,0


In [27]:
final.loc[final['Date'] == '2024-09-30']['Batter_name'].unique()

array(['Brandon Nimmo', 'Jorge Soler', 'Ramón Laureano', 'Gio Urshela',
       'Sean Murphy', 'Starling Marte', 'Pete Alonso', 'Tyrone Taylor',
       'Harrison Bader', 'Luis Torrens', 'Jose Iglesias', 'Orlando Arcia',
       'Luisangel Acuña', 'Marcell Ozuna', 'Ozzie Albies', 'Eddy Alvarez',
       'Michael Harris II', 'Matt Olson', 'Eli White', "Travis d'Arnaud",
       'Mark Vientos', 'J.D. Martinez', 'Francisco Alvarez',
       'Francisco Lindor', 'Jarred Kelenic'], dtype=object)

In [200]:
rbis_df = test.loc[(test['Rbi'] >= 1) & (test['Scoring_pos'] == 1)]
rbis_df

,Outcome,Rbi,Bases,Batter_id,Batter_name,Pitcher,Isout,Outs,homeScore,awayScore,inning,isTopInning,Date,Current_outs,Scoring_pos,player_position
433,single,1,RISP,457763,Buster Posey,vs_LHP,False,1,0,1,1,True,2014-04-01,1,1,2
7040,single,1,RISP,457763,Buster Posey,vs_RHP,False,1,3,1,3,False,2014-04-08,1,1,2
9759,single,1,RISP,457763,Buster Posey,vs_RHP,False,2,2,4,3,False,2014-04-10,2,1,2
16340,single,1,RISP,457763,Buster Posey,vs_LHP,False,1,1,0,3,False,2014-04-16,1,1,2
31000,single,1,RISP,457763,Buster Posey,vs_RHP,False,2,3,0,7,False,2014-04-30,2,1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157386,double,1,RISP,457763,Buster Posey,vs_LHP,False,2,1,3,1,False,2021-09-04,2,1,2
160468,double,1,RISP,457763,Buster Posey,vs_RHP,False,0,0,2,1,True,2021-09-07,0,1,2
175884,double,1,RISP,457763,Buster Posey,vs_RHP,False,0,1,6,7,True,2021-09-22,0,1,2
177772,single,1,RISP,457763,Buster Posey,vs_RHP,False,1,2,7,7,True,2021-09-24,1,1,2


In [44]:
scoring_pos = test[(test['Scoring_pos'] == 1)]
scoring_pos

,Outcome,Rbi,Bases,Batter_id,Batter_name,Pitcher,Isout,Outs,homeScore,awayScore,inning,isTopInning,runner_pos,Date,Current_outs,Scoring_pos
433,single,1,RISP,457763,Buster Posey,vs_LHP,False,1,0,1,1,True,"[0, 2, 1]",2014-04-01,1,1
2384,intent_walk,0,RISP,457763,Buster Posey,vs_RHP,False,2,5,4,8,True,"[0, 2]",2014-04-03,2,1
4516,field_out,0,RISP,457763,Buster Posey,vs_RHP,True,1,2,7,9,True,"[0, 2, 1]",2014-04-05,0,1
4464,field_out,0,RISP,457763,Buster Posey,vs_LHP,True,3,1,1,3,True,"[0, 3]",2014-04-05,2,1
7040,single,1,RISP,457763,Buster Posey,vs_RHP,False,1,3,1,3,False,"[0, 3, 1]",2014-04-08,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
175884,double,1,RISP,457763,Buster Posey,vs_RHP,False,0,1,6,7,True,"[0, 2, 1]",2021-09-22,0,1
177738,strikeout,0,RISP,457763,Buster Posey,vs_RHP,True,3,2,2,3,True,"[0, 2, 1]",2021-09-24,2,1
177772,single,1,RISP,457763,Buster Posey,vs_RHP,False,1,2,7,7,True,"[0, 3, 1]",2021-09-24,1,1
183945,sac_fly,1,RISP,457763,Buster Posey,vs_LHP,True,1,2,3,3,False,"[0, 3, 1]",2021-09-30,0,1


do one jsut for risp and do for all general atbats  /do langrage and machine learning

In [45]:
date = '2019-07-10'

In [46]:
current_events = scoring_pos.loc[scoring_pos['Date'] < date].tail(30)

In [47]:
current_events

,Outcome,Rbi,Bases,Batter_id,Batter_name,Pitcher,Isout,Outs,homeScore,awayScore,inning,isTopInning,runner_pos,Date,Current_outs,Scoring_pos
31430,single,1,RISP,457763,Buster Posey,vs_RHP,False,2,2,1,9,False,"[0, 2, 1]",2019-05-01,2,1
31370,field_out,0,RISP,457763,Buster Posey,vs_LHP,True,2,1,0,1,False,"[0, 3]",2019-05-01,1,1
32598,field_out,0,RISP,457763,Buster Posey,vs_RHP,True,2,0,0,1,True,"[0, 2]",2019-05-03,1,1
35373,home_run,3,RISP,457763,Buster Posey,vs_RHP,False,2,4,4,6,True,"[0, 2, 1]",2019-05-05,2,1
47409,field_out,0,RISP,457763,Buster Posey,vs_RHP,True,2,0,0,3,True,"[0, 3, 1]",2019-05-17,1,1
48627,single,1,RISP,457763,Buster Posey,vs_RHP,False,0,0,3,3,True,"[0, 3]",2019-05-18,0,1
48610,strikeout,0,RISP,457763,Buster Posey,vs_RHP,True,1,0,0,1,True,"[0, 2, 1]",2019-05-18,0,1
51547,grounded_into_double_play,0,RISP,457763,Buster Posey,vs_RHP,True,3,1,1,5,False,"[0, 2, 1]",2019-05-21,2,1
52940,single,1,RISP,457763,Buster Posey,vs_LHP,True,2,1,6,4,False,"[0, 2]",2019-05-22,1,1
52915,field_out,0,RISP,457763,Buster Posey,vs_LHP,True,3,0,0,1,False,"[0, 2]",2019-05-22,2,1


In [48]:
rbis = (current_events['Rbi'] != 0).sum()

successful = rbis / len(current_events)

In [49]:
successful = rbis / len(current_events)

In [50]:
successful

0.3

In [101]:
final['Outcome'].unique()

array(['strikeout', 'home_run', 'field_out', 'single', 'double', 'walk',
       'grounded_into_double_play', 'force_out', 'sac_fly', 'intent_walk',
       'sac_bunt', 'double_play', 'triple', 'hit_by_pitch', 'field_error',
       'pickoff_1b', 'sac_fly_double_play', 'fielders_choice',
       'catcher_interf', 'fielders_choice_out', 'caught_stealing_2b',
       'strikeout_double_play', 'pickoff_caught_stealing_2b',
       'caught_stealing_3b', 'pickoff_2b', 'other_out', 'wild_pitch',
       'pickoff_caught_stealing_3b', 'triple_play', 'runner_double_play',
       'pickoff_3b', 'caught_stealing_home',
       'pickoff_caught_stealing_home', 'balk', 'pickoff_error_1b',
       'sac_bunt_double_play', 'pitching_substitution', 'passed_ball',
       'ejection', 'stolen_base_2b', 'stolen_base_3b', 'pickoff_error_2b',
       'game_advisory', 'stolen_base_home', 'pickoff_error_3b'],
      dtype=object)

In [102]:
final['Bases'].unique()

array(['Empty', 'Men_On', 'RISP', 'Loaded'], dtype=object)